In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
prev = pd.read_csv(PATH + 'previous_application.csv')
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
prev.head(2)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0


In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
prev.columns.values

array(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
       'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT',
       'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT',
       'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
       'NAME_CASH_LOAN_PURPOSE', 'NAME_CONTRACT_STATUS', 'DAYS_DECISION',
       'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE',
       'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
       'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'SELLERPLACE_AREA',
       'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT', 'NAME_YIELD_GROUP',
       'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
       'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION',
       'NFLAG_INSURED_ON_APPROVAL'], dtype=object)

In [4]:
ignore = ['SELLERPLACE_AREA', 'PRODUCT_COMBINATION']
not_ignore = [x for x in prev.columns if x not in ignore]
prev = prev[not_ignore]

In [5]:
merged_col = 'SK_ID_CURR'
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()

# minus
name1 = minus_name('DAYS_TERMINATION', 'DAYS_FIRST_DRAWING')
prev[name1] = minus(prev, 'DAYS_TERMINATION', 'DAYS_FIRST_DRAWING')
name2 = minus_name('DAYS_FIRST_DUE', 'DAYS_FIRST_DRAWING')
prev[name2] = minus(prev, 'DAYS_FIRST_DUE', 'DAYS_FIRST_DRAWING')
name3 = minus_name('DAYS_TERMINATION', 'DAYS_LAST_DUE')
prev[name3] = minus(prev, 'DAYS_TERMINATION', 'DAYS_LAST_DUE')
print('minus')

# deal with some cat feat
prev['FLAG_NAME_CASH_LOAN_PURPOSE'] = (prev['NAME_CASH_LOAN_PURPOSE']=='XNA') + (prev['NAME_CASH_LOAN_PURPOSE']=='XAP')*2
prev['FLAG_NAME_GOODS_CATEGORY'] = (prev['NAME_GOODS_CATEGORY'] == 'XNA') + 0
print('deal')

# ratio
numerator = [x for x in prev.columns if 'AMT' in x]
denominator = [x for x in prev.columns if 'DAYS' in x or 'AMT' in x] + [name1, name2, name3]
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in prev.columns):
            prev[ratio_name(x, y)] = ratio(prev, x, y)
print('ratio')
        
# one hot
categorical = ['NAME_CONTRACT_TYPE', 'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY', 'NAME_CONTRACT_STATUS',
               'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
               'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NFLAG_INSURED_ON_APPROVAL', 
               'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'WEEKDAY_APPR_PROCESS_START'] + \
              ['FLAG_NAME_CASH_LOAN_PURPOSE', 'FLAG_NAME_GOODS_CATEGORY']
for x in categorical:
    print(x)
    df = df.merge(one_hot(prev, merged_col, x), on=merged_col, how='left')
    
# count
df = df.merge(count(prev, merged_col, 'NAME_CONTRACT_TYPE'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print('count')

# count ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)

# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CASH_LOAN_PURPOSE', 'NAME_GOODS_CATEGORY'] + categorical
for x in set(prev.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(prev, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')
    
columns = []
for x in df.columns:
    tmp = 'prev_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns

'done'

minus
deal
ratio
NAME_CONTRACT_TYPE
FLAG_LAST_APPL_PER_CONTRACT
NFLAG_LAST_APPL_IN_DAY
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NFLAG_INSURED_ON_APPROVAL
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
WEEKDAY_APPR_PROCESS_START
FLAG_NAME_CASH_LOAN_PURPOSE
FLAG_NAME_GOODS_CATEGORY
count
onehot_NAME_CONTRACT_TYPE_Cash loans
onehot_NAME_CONTRACT_TYPE_Consumer loans
onehot_NAME_CONTRACT_TYPE_Revolving loans
onehot_NAME_CONTRACT_TYPE_XNA
onehot_FLAG_LAST_APPL_PER_CONTRACT_N
onehot_FLAG_LAST_APPL_PER_CONTRACT_Y
onehot_NFLAG_LAST_APPL_IN_DAY_0
onehot_NFLAG_LAST_APPL_IN_DAY_1
onehot_NAME_CONTRACT_STATUS_Approved
onehot_NAME_CONTRACT_STATUS_Canceled
onehot_NAME_CONTRACT_STATUS_Refused
onehot_NAME_CONTRACT_STATUS_Unused offer
onehot_NAME_PAYMENT_TYPE_Cash through the bank
onehot_NAME_PAYMENT_TYPE_Cashless from the account of the employer
onehot_NAME_PAYMENT_TYPE_Non-cash from your account
onehot_NAME_PAYMENT_T

'done'

In [6]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'prev_onehot_NAME_CONTRACT_TYPE_Cash loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Consumer loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Revolving loans',
       'prev_onehot_NAME_CONTRACT_TYPE_XNA',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_N',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_Y',
       'prev_onehot_NFLAG_LAST_APPL_IN_DAY_0',
       'prev_onehot_NFLAG_LAST_APPL_IN_DAY_1',
       'prev_onehot_NAME_CONTRACT_STATUS_Approved',
       ...
       'prev_DAYS_TERMINATION_minus_DAYS_LAST_DUE_mean',
       'prev_DAYS_TERMINATION_minus_DAYS_LAST_DUE_median',
       'prev_DAYS_TERMINATION_minus_DAYS_LAST_DUE_max',
       'prev_DAYS_TERMINATION_minus_DAYS_LAST_DUE_min',
       'prev_DAYS_TERMINATION_minus_DAYS_LAST_DUE_sum',
       'prev_AMT_APPLICATION_divide_DAYS_TERMINATION_minus_DAYS_LAST_DUE_mean',
       'prev_AMT_APPLICATION_divide_DAYS_TERMINATION_minus_DAYS_LAST_DUE_median',
       'prev_AMT_APPLICATION_divide_DAYS_TERMINATION_minus_DAYS_LAST_

(356255, 536)

In [7]:
df.to_pickle(PATH + 'inter/poscash2curr.pkl')
'done'

'done'